<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Principal Component Analysis (PCA)

---
Authors: _Kiefer Katovich (SF), Adam Jones, Steven Longstreet(DC)_

### Learning Objectives

After this lesson, you will be able to:

- Describe what principal component analysis (PCA) does and what it’s used for in data science.
- Understand key terms, including _principal components, eigenvalues, and eigenvectors_.
- Practice computing PCA manually (and automatically, with scikit-learn).
- Represent PCA graphically and interpret results.

### Lesson Guide
- [Motivation](#motivation)
- [What is PCA?](#what)
    - [Eigenvalues and Eigenvectors](#eigenpairs)
    - [Principal Components](#pcs)
- [Manual PCA Code Along](#manual)
    - [1) Basic EDA](#basic-eda)
    - [2) Subset and Normalize](#subset)
    - [3) Find the Correlation Matrix](#corr)
    - [4) Eigenvalues and Eigenvectors](#eigen)
    - [5) Explained Variance](#var)
    - [6) Projection Matrix W](#projection)
    - [7) Transformed Matrix Z](#transformed)
    - [8–12) Plot and Interpret Principal Components](#plot-components)
- [PCA in scikit-learn](#auto)
- [Example: Eigenfaces](#example)
- [Further Reading](#further)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D # registers the 3D projection, but is otherwise unused

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.datasets import fetch_lfw_people
from sklearn.datasets import load_diabetes
%matplotlib inline

sns.set_style('white')


# Principal Component Analysis

PCA isn't exactly full machine learning algorithm, but instead an unsupervised learning algorithm. It is often used to **preprocess** data before it goes into a supervised learning method. Traditionally it is used to solve problems involving too many features and multicolinearity. 

## Let's dig into how it works 
Suppose you have $p$ feature columns. The **first principal component** is a linear combination of all $p$ columns that accounts for the **maximum variance** among them.  That is,

$$ z_1 = c_{11}x_1 + c_{12}x_2 + c_{13}x_3 + ... c_{1n}$$

The **second principal component** is another linear combination of the $p$ features that accounts for the maximum of the _remaining_ variance after the first. Another condition is that the second PC must be **orthogonal (perpendicular)** to the first.

$$ z_2 = c_{21}x_1 + c_{22}x_2 + c_{23}x_3 + ... c_{2n}$$

The **third principal component** maximizes the remaining variance while being orthogonal (read: _independent_) to the first two, and so on.


$$ z_3 = c_{31}x_1 + c_{32}x_2 + c_{33}x_3 + ... c_{3n}$$
$$...$$
$$ z_i = c_{i1}x_1 + c_{i2}x_2 + c_{i3}x_3 + ... c_{in}$$

![MachineLearning](assets/PCA.png)

Hands on visuals are a great way to learn. Here is an interactive website to help you understand how [PCA works](http://setosa.io/ev/principal-component-analysis/)

<a id="motivation"></a>
## Motivation

<b>Dimensionality reduction</b> <i>intelligently</i> decreases the number of random variables that will be considered for analysis -- leaving just the most important ones.

- Dimensionality reduction is not an end goal in itself but is instead a tool to form a data set with optimized features for further visualization and/or modeling.

> <b>Recall</b> that <i>collinear</i> variables are variables that can be linearly predicted from other variables with a substantial degree of accuracy.
> - Eliminating <b>redundant features</b> such as these leaves us with a smaller dataset, with minimal information lost.

#### How can we detect collinearity in our dataset?
To get a quick summary of our data, we can calculate a <b><i>covariance matrix</i></b> — an unstandardized correlation matrix.

$$\large
\begin{align}
Var(X) & = \begin{bmatrix}Var(X_1) & Cov(X_1,X_2)\\Cov(X_2,X_1) & Var(X_2)\end{bmatrix} \\
& = \begin{bmatrix}2&1\\1&4\end{bmatrix}
\end{align}
$$
+ The diagonal elements in a covariance matrix show us the variance of each of our features.
+ The off-diagonal elements show the covariance, i.e. the amount of collinearity (redundancy) between our variables.

#### What would an “ideal” covariance matrix look like?

---

An ideal covariance matrix for data would have <b>large numbers (variances) along the diagonal</b>, which would indicate a large amount of signal in the data. 
- It would also have <b>small values (or zeros) in the off-diagonal elements</b>, as these values indicate redundancy across our variables.

#### What if we DO have redundant features?

----

What can we do to try to remove any redundancies *and* preserve the signal?

- Surprise! We can use <i><b>PCA</b></i>!

<a id="what"></a>
## What is Principal Component Analysis (PCA)?

---

We've seen previously how we can use tools like <i>random forests</i> to accomplish "<b>feature <i>selection</i></b>". 
- Here, we'll focus on a form of dimensionality reduction known as <i>principal component analysis</i> that accomplishes "<b>feature <i>extraction</i></b>".

Whereas <b>feature <i>selection</i></b> attempts to reveal the subsets of the original data that are most relevant, <b>feature <i>extraction</i></b> combines features of a potentially correlated dataset resulting in a new <i>non-correlated</i> dataset with <i>fewer features</i>.

PCA is currently among the most popular dimensionality reduction algorithms.

#### TL;DR:
<i>Dimensionality reduction</i> is the process of combining or collapsing the existing features (columns in $X$) into fewer features. 

When we perform dimensionality reduction, we hope to:

- Retain the signal in the original data.
- Reduce noise.

PCA finds the linear combinations of the current predictor variables that create new variables, referred to as the "principal components". 

<center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/GaussianScatterPCA.svg/1024px-GaussianScatterPCA.svg.png' alt='pca' width='350'></center>

<div style="text-align: right"><a href="//commons.wikimedia.org/wiki/User:Nicoguaro" title="User:Nicoguaro">Source</a></div>

- The principle components are ranked in terms of how well they explain the variance in your original predictors.

A more natural way of thinking about PCA is that <b>it transforms the coordinate system so that the axes become the most concise, informative descriptors of our data as a whole.</i>

> The old axes are the original variables (columns).  
> The new axes are the principal components from PCA.

<center><img src='assets/pca_diagram.png' alt='pca diagram' width='1000'></center>

#### Additional resources
- [A great paper on PCA](http://arxiv.org/pdf/1404.1100.pdf)  
- [Interactive PCA plot](https://setosa.io/ev/principal-component-analysis/)  
- [An nice example of performing PCA](http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda)

### Why Would We Want to Perform PCA?

- We can <b><i>reduce the number of dimensions</i></b> (remove less important components) while retaining <i>most</i> of our signal (and removing <i>mostly</i> noise).
- Because we’re assuming our variables are inter-related (at least in the sense that, together, they explain a dependent variable), the information of interest should exist along directions with the largest variance.
    - As a result, the directions of largest variance should have the highest signal-to-noise ratio.
- Correlated predictor variables (also referred to as redundancy of information) are combined into new, independent variables. 
    - Our predictors from PCA are <i>guaranteed to be independent</i>.

### But first, a few examples...

> #### First, a descriptive example
> <center><img src='https://upload.wikimedia.org/wikipedia/commons/9/90/PCA_fish.png' alt='pca on fish' width='500'></center>
> <div style="text-align: right"><a href="https://fr.wikipedia.org/wiki/Utilisateur:Lehalle">Source</a></div>
>
> In this example, the two axes represented are the best way to represent 3-dimensional data in a two-dimensional plane, with the least possible loss of the original variance.
>
> Let's look at a similar example in Python.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# create data
n = 50
xs = np.random.normal(0, 5, n)
ys = np.random.normal(0, 20, n)
zs = np.random.normal(0, 5, n)

# create plot
ax.scatter(xs, ys, zs)
ax.set_xlabel('X', fontsize=14)
ax.set_ylabel('Y', fontsize=14)
ax.set_zlabel('Z', fontsize=14)
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.set_zlim(-40, 40)

###################################
angle = 0 # try changing to zero #
###################################

# rotate the axes and show
ax.view_init(30, angle)
plt.show();

#### A more applied example

<center><img src='http://www.scholarpedia.org/w/images/thumb/a/a6/QQ_Fig1.jpg/400px-QQ_Fig1.jpg' alt='pca' width='500'></center>

<div style="text-align: right"><a href="http://www.scholarpedia.org/article/Spike_sorting">Source</a></div>

<i><b>Spike sorting</b></i> is the grouping of spikes into clusters based on the similarity of their shapes. 
- Since each neuron tends to exhibit spikes of a particular shape, the resulting clusters correspond to the activity of different neurons. 
- The result of spike sorting is the determination of which spike corresponds to which of these neurons.

<img style="float: right" src='./assets/spike_sorting.jpg' alt='spike sorting' width='400'/>

> #### How do we sort the spikes? 
> 
> One common method is to extract features of the spike shapes. 
> - But, the <i>major challenge</i> is how to select which are the best features!
> - One (very popular) method for choosing features automatically is with PCA. 
> - The idea behind PCA is to find an ordered set of vectors that capture the directions containing the <i>greatest variation</i> in the data. 
> - The input data are the original spikes, and the output data are those same spikes, but with fewer dimensions.
>

<b>Our final example</b> of how PCA can used is <i>reducing the statistical complexity of face images</i> (i.e. calculating <i><b>'eigen-faces'</b></i>).
- PCA is performed on a large set of images depicting different human faces. 
- Eigenfaces can be considered a set of "standardized face ingredients", derived from statistical analysis of many pictures of faces.
- See [here](https://en.wikipedia.org/wiki/Eigenface) for more info.

<center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Eigenfaces.png/220px-Eigenfaces.png' alt='eigenfaces' width='300'></center>

### The Process of PCA 

---

Say we have a matrix, $X$, of predictor variables. 
- PCA gives us the ability to transform our $X$ matrix into a new matrix, $Z$.

<br></br>
<center><img src='./assets/transformed_xyz.png' alt='3D transform' width='850'></center>
<div style="text-align: right"><a href="http://phdthesis-bioinformatics-maxplanckinstitute-molecularplantphys.matthias-scholz.de/">Source</a></div>


+ First, we’ll derive a <b>weighting matrix</b>, $W$, from the correlational/covariance structure of $X$. 
    - This allows us to perform the transformation.

+ Each successive dimension (column) in $Z$ will be ranked and ordered according to the variance in its values.

### Assumptions

1. **Linearity:** The data do not hold nonlinear relationships.
    - e.g.: 'hours worked' vs. '$ paid' when including overtime.
    
    
2. **Large variances define importance:** The dimensions are constructed to maximize the remaining variance. 
    - We assume any orthogonal variance is due to noise rather than signal.

> <b>Yes!</b> #2 does mean it is <i>necessary to normalize data before performing PCA</i>. 
> - PCA calculates a new projection of your data set, and the new axis are based on the standard deviation of your variables.

The resulting principal components (columns of $Z$) will be uncorrelated. 
- This also makes PCA a useful pre-processing step for algorithms requiring uncorrelated input features.

> <b>Yes!</b> This does mean that the <i>maximum number of principal components is the number of features</i>. 

<a id="eigenpairs"></a>
### Eigenvalues and Eigenvectors

#### Eigenvectors

<center><img src='./assets/eigenvectors_orthogonal.png' alt='orthogonal vectors' width='700'></center>

An eigenvector specifies a direction through the original coordinate space.
- The eigenvector with the highest corresponding eigenvalue is the first principal component (PC).

<center><img src='./assets/transformed_xy.png' alt='transformed vectors' width='700'></center>

> In some sense, the first PC is just a line fitted to the data.
> - It is the <i>linear surface</i> (hyperplane) that is <b>closest</b> to the $n$ observations.
> - Such a line will likely provide a good summary of the data.

<center><img src='./assets/eigenvalue.png' alt='eigen values' width='700'></center>

#### Eigenvalues

Eigenvalues indicate the amount of variance in the direction of its corresponding eigenvector. 
- The larger the eigenvalue, the more variance (information) in our data its corresponding eigenvector explains.

> #### Every eigenvector has a corresponding eigenvalue.
> These are sometimes called <b>eigenpairs</b>.

<center><img src='./assets/pca_coord_trans.png' alt='pca transformations' width='1200'></center>

<a id="pcs"></a>
### Principal Components

#### <i>Principal components</i> are the vectors that define the new coordinate system for your data.
- Transforming your original data columns onto the principal component (PC) axes constructs new variables that explain as much variance as possible and are independent (uncorrelated).

Creating these variables is a well-defined mathematical process. 
- In essence, <b>each component is created as a weighted sum of your original columns, such that all components are orthogonal (perpendicular) to each other.</b>

- [Explained Visually](http://setosa.io/ev/) has a nice [interactive visualization for PCA](http://setosa.io/ev/principal-component-analysis/).

<center><img src='./assets/setosa_pc1.png' alt='setosa transformations' width='1000'></center>

<center><b>Optional intermission</b></center>

<a id="manual"></a>
## Manual PCA Code Along

**PCA Steps**

1. **Standardize data:** Centering is required. Full normalization is nice for future visuals.
2. **Calculate eigenvectors and eigenvalues:** Do this from the correlation or covariance matrix.
3. **Sort eigenvalues:** Choose eigenvectors that correspond to the largest eigenvalues. The number you choose is up to you, but we will take two for the sake of visualization here.
4. **Construct the projection weighting matrix, $W$:** Do this from the eigenvectors.
5. **Transform the original data set, $X$, with $W$:** Obtain the new two-dimensional transformed matrix, $Z$.

**Data**

We're going to be using a simple 75-row, four-column data set with generic demographic information. It contains:

- `Age` (limited to 20–65)
- `Income`
- `Health` (A rating on a scale from 1:100, where 100 is the best health)
- `Stress` (A rating on a scale from 1:100, where 100 is the most stressed)

All of the variables are continuous.

In [ ]:
# load the data
demo_data = pd.read_csv('./datasets/simple_demographics.csv')

<a id="basic-eda"></a>
### 1) Basic EDA.

Create a Seaborn regplot for each:

1. `Age` versus `income`.
2. `Age` versus `health`.
3. `Age` versus `stress`.

Also make a pair plot of the entire data set.

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.gca()

ax = sns.regplot('age', 'income', data=demo_data, fit_reg=False, scatter_kws={'s':50}, ax=ax)

ax.set_ylabel('income', fontsize=15)
ax.set_xlabel('age', fontsize=15)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax.set_title('age vs income\n', fontsize=18)

plt.show()

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.gca()

ax = sns.regplot('age', 'health', data=demo_data, fit_reg=False, scatter_kws={'s':50}, ax=ax)

ax.set_ylabel('health', fontsize=15)
ax.set_xlabel('age', fontsize=15)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax.set_title('age vs health\n', fontsize=18)

plt.show()

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.gca()

ax = sns.regplot('age', 'stress', data=demo_data, fit_reg=False, scatter_kws={'s':50}, ax=ax)

ax.set_ylabel('stress', fontsize=15)
ax.set_xlabel('age', fontsize=15)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax.set_title('age vs stress\n', fontsize=18)

plt.show()

In [ ]:
# alternative, showing all possible pairs
sns.pairplot(demo_data);

<a id="subset"></a>
### 2) Subset and normalize.

Subset the data to only include:

- `Income`  
- `Health`  
- `Stress`

We'll be comparing the principal components to `age` specifically, so we're leaving it out here.

In [ ]:
# subset data
data_noage = demo_data[['health','income','stress']]

# scale variables
data_noage = (data_noage - data_noage.mean()) / data_noage.std()

<a id="corr"></a>
### 3) Calculate the correlation matrix.

We'll be using the correlation matrix to calculate the eigenvectors and eigenvalues.

In [ ]:
# extract correlation matrix
data_noage_corr = np.corrcoef(data_noage.values.T)
data_noage.corr()

<a id="eigen"></a>
### 4) Calculate the eigenvalues and eigenvectors from the correlation matrix.

NumPy has a convenient function to calculate this:

`eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)`

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(data_noage_corr)

print(f'Eigenvalues: {eig_vals}\n')
print(f'Eigenvectors: {eig_vecs}')

<a id="var"></a>
### 5) Calculate and plot the explained variance.

A useful measure is the <b>explained variance</b>, which is calculated from the eigenvalues. 

The explained variance tells us how much information (variance) is captured by each principal component:

$$\large ExpVar_i = \bigg(\frac{eigenvalue_i}{\sum_j^n{eigenvalue_j}}\bigg) * 100$$

In [ ]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

print(f'Cumulative Explained Variance: {cum_var_exp}')

In [ ]:
# let's define some functions to plot out the explained var. for each component

component_number = [1,2,3]

def scree_plot(ax):
    # scree plot
    ax.plot(component_number, var_exp, lw=7)
    ax.axhline(y=0, linewidth=3, color='grey', ls='dashed')
    ax.set_xlim([1,3])
    ax.set_ylim([-5,105])
    ax.set_ylabel('% variance explained', fontsize=16)
    ax.set_xlabel('component', fontsize=16)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_xticks(component_number, minor=False)
    ax.set_title('scree plot', fontsize=20)

def cum_scree_plot(ax):
    # cumulative scree plot
    ax.plot(component_number, cum_var_exp, lw=7)
    ax.axhline(y=100, linewidth=3, color='grey', ls='dashed')
    ax.set_xlim([1,3])
    ax.set_ylim([-5,105]) 
    ax.set_ylabel('cumulative % variance explained', fontsize=16)
    ax.set_xlabel('component', fontsize=16)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_xticks(component_number, minor=False)
    ax.set_title('cumulative scree plot', fontsize=20)

In [ ]:
plt.figure(figsize=(14,7))

ax = plt.subplot(121)
scree_plot(ax)
ax = plt.subplot(122)
cum_scree_plot(ax)

plt.show()

<a id="projection"></a>
### 6) Construct the projection matrix, $W$.

This is simply a matrix of our top $n$ (two, in this case) eigenvectors.

The eigenvectors are concatenated as columns.

1) Start by ordering the eigenvectors from largest to smallest by their corresponding eigenvalues.
- Concatenate the eigenvectors together. `np.hstack()` is useful for this.

In [ ]:
# sort eigenvectors by corresponding eigenvalues (largest to smallest)
value_vector_pairs = [[eig_vals[i], eig_vecs[:,i]] for i in range(len(eig_vals))]
# value_vector_pairs = list(zip(eig_vals, eig_vecs.T)) # also works
value_vector_pairs.sort(reverse=True)
value_vector_pairs

In [ ]:
weight_2d_projection = np.hstack((value_vector_pairs[0][1].reshape(-1,1),
                                  value_vector_pairs[1][1].reshape(-1,1)))

print(f'Weight data 2D PCA projection matrix:\n{weight_2d_projection}')

<a id="transformed"></a>
### 7) Construct the transformed two-dimensional matrix, $Z$.

To do this, we take the <i>matrix product\*</i> of our three-dimensional demographic matrix, $X$, with the projection matrix, $W$.

<div style="text-align: right"><i>*One of these again!?</i></div>

In [ ]:
# recall our input
data_noage.head()

In [ ]:
# mat mult X with W to get Z
Z = data_noage.dot(weight_2d_projection)
Z.head()

<a id="plot-components"></a>
### 8) Plot Principal Component 1 (PC1) vs. Principal Component 2 (PC2).

Principal Component 1 is the first column in $Z$ and Principal Component 2 is the second.
- Notice how they are uncorrelated.

In [ ]:
plt.figure(figsize=(7,5))
ax = plt.gca()
ax = sns.regplot(Z.iloc[:,0], Z.iloc[:,1], fit_reg=False, scatter_kws={'s':50}, ax=ax)

ax.set_xlabel('principal component 1', fontsize=15)
ax.set_ylabel('principal component 2', fontsize=15)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax.set_title('PC1 vs PC2', fontsize=18)

plt.show()

### 9) Plot age vs. PC1 with `regplot`

Notice how tight the relationship is. Principal Component 1 took the shared variance out of income, health, and stress, which are directly related to increasing age. 

This principal component, or more specifically the column-weighting matrix $W$, is essentially <b>capturing the latent age variance embedded in these variables.</b>

In [ ]:
plt.figure(figsize=(7,5))
ax = plt.gca()
ax = sns.regplot(Z.iloc[:,0], demo_data.age.values,
                 fit_reg=True, scatter_kws={'s': 50}, ax=ax)

ax.set_xlabel('principal component 1', fontsize=15)
ax.set_ylabel('age', fontsize=15)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax.set_title('PC1 vs age', fontsize=18)

plt.show()

### 10) Concatenate PC1 and PC2 to the full demographic (four-dimensional) data set, then melt it with PC1, PC2, and the index variables.

1. Renormalize so that all four variables are on the same scale.
2. Remember the Pandas melt code:

```python
melted_df = pd.melt(df, id_vars=['PC1','PC2'])
```

In [ ]:
demo_data.head()

In [ ]:
# copy data
demo_pcs = demo_data.copy()

# scale data
demo_pcs = (demo_data - demo_data.mean()) / demo_data.std()

# add transformed data 
demo_pcs['PC1'] = Z.iloc[:,0]
demo_pcs['PC2'] = Z.iloc[:,1]

# convert from wide to long format
demo_pcs = pd.melt(demo_pcs, id_vars=['PC1','PC2'])

demo_pcs.sample(10) 

### 11) Use `lmplot` to view PC1 versus all four variables.

Make the `col` keyword argument and the `hue` keyword argument "variable,” assuming that's what you called them in the melt command (those are the defaults).

Make `col_wrap = 2` and `size = 7`, or something similar, to make it visually appealing.  

In [ ]:
pc1 = sns.lmplot(x="PC1", y="value", col="variable", hue="variable", 
                 data=demo_pcs, col_wrap=2, scatter_kws={'s': 50})

### 12) Use `lmplot` to do the same for PC2.

Notice how PC2 captures the variance of income, which was not captured well by PC1. 
- This makes sense, as the variance each principal component captures has to be orthogonal to the other components.

In [ ]:
pc2 = sns.lmplot(x="PC2", y="value", col="variable", hue="variable", 
                 data=demo_pcs, col_wrap=2, scatter_kws={'s':50})

<a id="auto"></a>
## PCA in scikit-learn

In [ ]:

pca = PCA()
X_r = pca.fit_transform(data_noage)

In [ ]:
ratios = pca.explained_variance_ratio_
print(ratios)

In [ ]:
print(pca.components_)

In [ ]:
# this is only two components, column-by-column of the above
weight_2d_projection

## Using Real Data

Let's work with the cancer data set again since it had so many features.

In [ ]:

cancer = load_breast_cancer()
cancer.keys()

In [ ]:
# If it's been awhile since you used a toy dataset - check out the description
print(cancer['DESCR'])

In [ ]:
X = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])
#(['DESCR', 'data', 'feature_names', 'target_names', 'target'])
ydf = pd.DataFrame(cancer['target'],columns=['malignant'])

#Combined the dataframes
df=pd.concat([X,ydf],axis=1)

df.head()

## PCA Visualization

As we've noticed before it is difficult to visualize high dimensional data, we can use PCA to find the first two principal components, and visualize the data in this new, two-dimensional space, with a single scatter-plot. Before we do this though, we'll need to scale our data so that each feature has a single unit variance.

In [ ]:

scaler = StandardScaler()
scaler.fit(X)

scaled_data = scaler.transform(X)

PCA with Scikit Learn uses a very similar process to other preprocessing functions that come with SciKit Learn. We instantiate a PCA object, find the principal components using the fit method, then apply the rotation and dimensionality reduction by calling transform().

We can also specify how many components we want to keep when creating the PCA object.

In [ ]:

pca = PCA(n_components=2)
pca.fit(scaled_data)

#Now we can transform this data to its first 2 principal components

x_pca = pca.transform(scaled_data)

print(scaled_data.shape)
print(x_pca.shape)

In [ ]:
feat_import=pd.DataFrame(pca.components_,columns=cancer['feature_names'],index = ['PC-1','PC-2'])
feat_import.T

In [ ]:
# Now that we've reduced our model to two dimensions - lets look at it
plt.figure(figsize=(8,6))
plt.scatter(x_pca[:,0],x_pca[:,1],c=cancer['target'],cmap='plasma')
plt.xlabel('First principal component')
plt.ylabel('Second Principal Component')

Clearly by using these two components we can easily separate these two classes.

## Interpreting the components 

Unfortunately, with this great power of dimensionality reduction, comes the cost of being able to easily understand what these components represent.

The components correspond to combinations of the original features, the components themselves are stored as an attribute of the fitted PCA object:

In [ ]:
pca.components_

In [ ]:
sorted(list(zip(pca.components_[0],df.columns)))

In this numpy matrix array, each row represents a principal component, and each column relates back to the original features. Fir example, here is the first component:

$$z_1 = 0.21890244x_1 + 0.10372458x_2 + 0.22753729x_3 + 0.22099499x_4 + 0.14258969x_5 + ... + 0.13178394x_{30}$$

we can visualize this relationship with a heatmap:

In [ ]:
df_comp = pd.DataFrame(pca.components_,columns=cancer['feature_names'])
plt.figure(figsize=(12,6))
sns.heatmap(df_comp,cmap='plasma',)

This heatmap shows the how each variable contributes to each of our two principle components.

## What's the right number of n_components?

Finding the right number with PCA can be accomplished by applying it to a model and seeing where it best maximizing it's score

In [ ]:

# How does it look with the whole feature set?

lg=LogisticRegression()
lg.fit(scaled_data,ydf)
lg.score(scaled_data,ydf)

In [ ]:
# How about reducing it to our two pca features?
lg=LogisticRegression()
lg.fit(x_pca,ydf)
lg.score(x_pca,ydf)

#### What does this tell us?

...


## Introducing Pipeline

To find the right value of n_components we could cycle through this a few times. However - to save us some time lets introduce the sklearn package - [Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

Pipeline sequentially applies a list of transforms and passes them to a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.

Included **Methods**

| Method                     | Application   |
|:---------------------------|------------:|
|decision_function(X)        |	Apply transforms, and decision_function of the final estimator|
|fit(X[, y])                 |	Fit the model|
|fit_predict(X[, y])         |	Applies fit_predict of last step in pipeline after transforms.|
|fit_transform(X[, y])       |	Fit the model and transform with the final estimator|
|get_params([deep])          |	Get parameters for this estimator.|
|predict(X)                  |	Apply transforms to the data, and predict with the final estimator|
|predict_log_proba(X)        |	Apply transforms, and predict_log_proba of the final estimator|
|predict_proba(X)            |	Apply transforms, and predict_proba of the final estimator|
|score(X[, y, sample_weight])|	Apply transforms, and score with the final estimator|
|set_params(**kwargs)        |	Set the parameters of this estimator.|

In [ ]:
# Optimizing for variance of n_components could take awhile. 
# Let's make our coding easier by putting all these models into a pipeline

pipe = Pipeline([
    ('sc', StandardScaler()),
    ('pc', PCA(n_components=2)),
    ('lg', LogisticRegression())
])

#### What will the Standard Scalar do to our data before we fit each model?

...

In [ ]:
pipe.fit(X,ydf)

pipe.score(X,ydf)

In [ ]:
#What's our model look like?
pipe.get_params

#### We can use the train/test split procedure from the previous lesson to see how each of these 30 versions performs.

In [ ]:

X_train, X_test, y_train, y_test=train_test_split(X,ydf, random_state=1)

# This for loop goes through every number of possible components and tests the accuracy of a model fitted to that many components.

acc_list = []
k_range = range(1,X.shape[1] + 1)
for k in k_range:
    pipe.set_params(pc__n_components=k)
    pipe.fit(X_train, y_train)
    acc = pipe.score(X_test, y_test)
    acc_list.append(acc)
    print(f"k = {k}: Acc = {acc}")

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(acc_list);
plt.ylabel('Accuracy - higher is better')
plt.xlabel('Number of Components Included');

### Which number of n_components would you choose? why?

#### Hint: what were we trying to reduce with PCA?

In [ ]:
# we can visualize the list
k_list=dict(zip(k_range,acc_list))
k_list

In [ ]:
# We can also find the greatest variance explained (for the first time)

print(sorted(k_list.items(), key=lambda x: (x[1]), reverse=True)[0])
print(sorted(k_list.items(), key=lambda x: (x[1]), reverse=True)[0][0])
best_k=sorted(k_list.items(), key=lambda x: (x[1]), reverse=True)[0][0]

In [ ]:
y = ydf
#What happens when we add that back into our pipe?
pipe_best_k = Pipeline([
    ('sc', StandardScaler()),
    ('pc', PCA(n_components=14)),
    ('lg', LogisticRegression())
])
pipe_best_k.fit(X,y)
pipe_best_k.score(X,y)

## Instead of setting n_components

Another approach to PCA is setting it against the amount of variance you want to explain

In [ ]:
# The first approch is to see how many component explain variance. After the first 4... not so much
pca_graph=PCA().fit(cancer.data)
plt.plot(np.cumsum(pca_graph.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
# Let's set a model that'll explain 95% of variance
pca=PCA(.95)
pca.fit(X_train)
pca.n_components_

### Hmmm... the boston dataset wasn't a great example. Let's find one a bit more complex

In [ ]:
# Let's look across other toy datasets - diabetes is more complex

diabetes = load_diabetes()


In [ ]:
pca_graph=PCA().fit(diabetes.data)
plt.plot(np.cumsum(pca_graph.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
#Ok - Bigger increase
pca=PCA(.94)
pca.fit(diabetes.data)
pca.n_components_

<a id="example"></a>
## Eigenfaces: Let's kick it up a notch with something a bit more complex


<div style="text-align: right"><a href="https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html#Example:-Eigenfaces">Source</a></div>

In [ ]:

faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Because this is a large dataset, we will use RandomizedPCA—it contains a randomized method to approximate the first N principal components much more quickly than the standard PCA estimator, and thus is very useful for high-dimensional data (here, a dimensionality of nearly 3,000). We will take a look at the first 150 components:

In [ ]:
pca = PCA(n_components=150, svd_solver='randomized')
pca.fit(faces.data)

In this case, it can be interesting to visualize the images associated with the first several principal components (these components are technically known as "eigenvectors," so these types of images are often called "eigenfaces"). As you can see in this figure, they are as creepy as they sound:

In [ ]:
fig, axes = plt.subplots(3, 8, figsize=(9, 4),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap='bone')

The results are very interesting, and give us insight into how the images vary: for example, the first few eigenfaces (from the top left) seem to be associated with the angle of lighting on the face, and later principal vectors seem to be picking out certain features, such as eyes, noses, and lips. Let's take a look at the cumulative variance of these components to see how much of the data information the projection is preserving:

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

We see that these 150 components account for just over 90% of the variance. That would lead us to believe that using these 150 components, we would recover most of the essential characteristics of the data. To make this more concrete, we can compare the input images with the images reconstructed from these 150 components:

In [ ]:
# compute the components and projected faces
pca = PCA(n_components=150, svd_solver='randomized').fit(faces.data)
components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

In [ ]:
# plot the results
fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap='binary_r')
    
ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel('150-dim\nreconstruction');

The top row here shows the input images, while the bottom row shows the reconstruction of the images from just 150 of the ~3,000 initial features. This visualization makes clear why the PCA feature selection used in In-Depth: Support Vector Machines was so successful: although it reduces the dimensionality of the data by nearly a factor of 20, the projected images contain enough information that we might, by eye, recognize the individuals in the image. What this means is that our classification algorithm needs to be trained on 150-dimensional data rather than 3,000-dimensional data, which depending on the particular algorithm we choose, can lead to a much more efficient classification.

### Recap

- PCA is an unsupervised method used to reduce the dimensionality of data.
- It does so by taking a <i>linear</i> combination of features.
- It is common to plot the transformed data in PC space.
- PCA searches for the directions that have the largest variance in the data.
- All principal components are orthogonal to each other.


# Summary:

### What does it do?
* Creates *linearly independent* predictors
* Allows you to only use the most valuable features

### What are the components?
* Values calculated from the raw observations:

$$ z_1 = c_{11}x_1 + c_{12}x_2 + c_{13}x_3 + ... c_{1n}$$
$$ z_2 = c_{21}x_1 + c_{22}x_2 + c_{23}x_3 + ... c_{2n}$$
$$...$$
$$ z_i = c_{i1}x_1 + c_{i2}x_2 + c_{i3}x_3 + ... c_{in}$$

* $z_1$ is always the strongest predictor. 
* $z_2$ is always the strongest predictor that is completely independent from $z_1$.  
* $z_3$ is always the strongest predictor that is completely independent from $z_1$ and $z_2$.  
* We can keep this going until the number of compoenents equals the number of predictors

### Why do we exclude components from our model?
* If we included all components, we would be getting the exact same result as if we hadn't used PCA
* By excluding components we can avoid overfitting the model, we are essentially ignoring the information that is least reliable.

### Points to Remember
* PCA is used to overcome features redundancy in a data set.
* These features are low dimensional in nature.
* These features a.k.a components are a resultant of normalized linear combination of original predictor variables.
* These components aim to capture as much information as possible with high explained variance.
* The first component has the highest variance followed by second, third and so on.
* The components must be uncorrelated (remember orthogonal direction ? ). See above.
* Normalizing data becomes extremely important when the predictors are measured in different units.
* PCA works best on data set having 3 or higher dimensions. Because, with higher dimensions, it becomes increasingly difficult to make interpretations from the resultant cloud of data.
* PCA is applied on a data set with numeric variables.
* PCA is a tool which helps to produce better visualizations of high dimensional data.

<a id="more-reading"></a>
### More Useful Links, Reading, and References for Images

- [PCA 4 Dummies](https://georgemdallas.wordpress.com/2013/10/30/principal-component-analysis-4-dummies-eigenvectors-eigenvalues-and-dimension-reduction/)
- [Stack Overflow: Making Sense of PCA](http://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues)
- [PCA and Spectral Theorem](http://stats.stackexchange.com/questions/217995/what-is-an-intuitive-explanation-for-how-pca-turns-from-a-geometric-problem-wit)
- [PCA in Three Steps: Eigendecomposition and SVD](http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda)
- [Tutorial on PCA](http://arxiv.org/pdf/1404.1100.pdf)
- [PCA Math and Examples](http://www.stat.cmu.edu/~cshalizi/uADA/12/lectures/ch18.pdf)

#### Onwards!
... to the [lab](./practice/intro_to_PCA-lab.ipynb)!

<center><img src='https://www.publicdomainpictures.net/pictures/290000/velka/lab-doctor.jpg' alt='lab' width='400'></center>
